In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import os
import os.path
import pandas as pd
import numpy as np
import sys
import re
import sys
import json

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

pd.set_option('display.max_columns', 50)

In [7]:
from cig_script.config.config_env import Config
from cig_script.compare_and_checker import compare_total_and_cargo,_read_total_mail_df,_read_cargo_df

In [8]:
tot_one_car_sheet = Config().total_one_car_sheet_name
tot_mul_car_sheet = Config().total_mul_car_sheet_name

In [9]:
tot_one_car_df,tot_mul_car_df = _read_total_mail_df()
cargo_result_df = _read_cargo_df()
tot_mul_car_df

,YR,CONSIGNEE NAME:,ACID NO,EGYPTIAN IMPORTER VAT NUMBER,FOREIGN EXPORTER REGISTRATION NUMBER,MODEL,CHASSINO,gen_time
0,"{""1"": 2012.0, ""2"": 2011.0, ""3"": 2007.0, ""4"": 2...",TRANS WORLD TRADING CO. ( ELROTOB BROTHERS),2049062532022080021,204906253,6348801265,"{""1"": ""AVANTE"", ""2"": ""AVANTE"", ""3"": ""AVANTE"", ...","{""1"": ""KMHDG41DBCU498818"", ""2"": ""KMHDH41DBBU03...",2022-08-16 02:11
1,"{""1"": ""2012"", ""2"": ""2013"", ""3"": ""2011"", ""4"": ""...",TIBA INTERNATIONAL FOR READY-MADE CLOTHES /AMR...,2049459332022060331,204945933,5032125413,"{""1"": ""AVANTE"", ""2"": ""AVANTE"", ""3"": ""AVANTE"", ...","{""1"": ""KNAFX412BDA019546"", ""2"": ""KMHDG41DBDU81...",2022-08-16 02:06


In [10]:
cargo_mul_car_df = cargo_result_df.copy().loc[(cargo_result_df['car_count'] != 1),:]
cargo_mul_car_df

,m_bl,cnt,SHIPPER NAME,CONSIGNEE NAME,car_count,total_weight,total_cbm,MODEL,YR,CHASSINO,ACID_NO,FREIGHT_FORWARDER_ID,IMPORTER_TAX_NUMBER
2,AHSINPS2207A003,3,"SEJONG GENERAL TRADING CO.,LTD 192,NEUNGHEODAE...",TIBA INTERNATIONAL FOR READY-MADE CLOTHES/ AMR...,9,"27,865.0",198.477,"{""1"": ""AVANTE"", ""2"": ""AVANTE"", ""3"": ""AVANTE"", ...","{""1"": ""2015"", ""2"": ""2016"", ""3"": ""2013"", ""5"": ""...","{""1"": ""KNAFX412BDA019546"", ""2"": ""KMHDG41DBDU81...",2049459332022060331,5032125413,204945933
3,AHSINPS2207A004,4,STAR TRADING (WASEEM MUKHTAR TAX ID: 503-21-25...,TIBA INTERNATIONAL FOR READY-MADE CLOTHES /AMR...,7,"8,825.0",76.963,"{""1"": ""AVANTE"", ""2"": ""AVANTE"", ""3"": ""AVANTE"", ...","{""1"": ""2012.0"", ""2"": ""2013.0"", ""3"": ""2011.0"", ...","{""1"": ""KNAFX412BDA019546"", ""2"": ""KMHDG41DBDU81...",2049459332022060331,5032125413,204945933


In [11]:
def _json_to_list(json_format):
    _dict = json.loads(json_format)
    _list = []
    for key in _dict:
        _list.append(_dict[key])
        
    return _list

In [12]:
def _check_mul_chassino(cargo_mul_chassino,cargo_mul_acid,tot_mul_car_df):
    matching_possibilty = 0
    matched_df = pd.DataFrame()
    check_acid = False
    cargo_mul_chassino_list = _json_to_list(cargo_mul_chassino)
    
    
    tot_mul_acid_df = tot_mul_car_df.copy().loc[(tot_mul_car_df['ACID NO']==cargo_mul_acid),:]
    
    if tot_mul_acid_df.empty:
        # not matched
        check_acid = False
        
    
    elif len(tot_mul_acid_df) == 1:
        tot_mul_chassino = tot_mul_acid_df['CHASSINO'].values[0]
        tot_mul_chassino_list = _json_to_list(tot_mul_chassino)
    
        common_set_list = list(set(tot_mul_chassino_list) & set(cargo_mul_chassino_list))
        
        if len(common_set_list) == 0: #0% matching
            check_acid = False
        elif len(tot_mul_chassino_list) == len(common_set_list): #100% matching
            check_acid = True
        else : 
            check_acid = False #partially matching
        
    else:
        # ACID가 여러개 matching되었을 때,
        for idx in tot_mul_acid_df.index:
            tot_mul_chassino = tot_mul_acid_df._get_value(idx,'CHASSINO')
            tot_mul_chassino_list = _json_to_list(tot_mul_chassino)
            
            common_set_list = list(set(tot_mul_chassino_list) & set(cargo_mul_chassino_list))
        
            if len(common_set_list) > matching_possibilty:
                matching_possibilty = len(common_set_list)
                matched_df = tot_mul_acid_df.copy().loc[idx,:]
            
            if check_acid:
                pass
            else:
                if len(common_set_list) == 0:
                    check_acid = False #0% matching
                elif len(common_set_list) == matching_possibilty: 
                    check_acid = True 
                else:  
                    check_acid = False #0 Partially matching
    
        tot_mul_acid_df = matched_df

    return check_acid,tot_mul_acid_df

In [13]:
def _check_mul_con(idx,cargo_mul_car_df,match_df):
    cargo_mul_car_df = cargo_mul_car_df.astype(str)
    match_df = match_df.astype(str)
    
    cargo_mul_con = cargo_mul_car_df._get_value(idx,'CONSIGNEE NAME')
    tot_mul_con = match_df['CONSIGNEE NAME:'].values[0]
    return cargo_mul_con == tot_mul_con

In [14]:
def _duplicate_cnt_dict(_list):
    count = {}
    for i in _list:
        try: count[i] += 1
        except: count[i]=1
    return count

In [15]:
def _check_mul_model(idx,cargo_mul_car_df,match_df):
    cargo_mul_model_dict = {}
    tot_mul_model = {}
    match_cnt = 0
    tot_cnt = 0
    cargo_cnt = 0
    
    cargo_mul_car_df = cargo_mul_car_df.astype(str)
    match_df = match_df.astype(str)

    cargo_mul_model = cargo_mul_car_df._get_value(idx,'MODEL')
    tot_mul_model = match_df['MODEL'].values[0]
    
    cargo_mul_model_list = _json_to_list(cargo_mul_model)
    tot_mul_model_list = _json_to_list(tot_mul_model)
    
    cargo_mul_model_dict = _duplicate_cnt_dict(cargo_mul_model_list)
    tot_mul_model_dict = _duplicate_cnt_dict(tot_mul_model_list) 

    for key in cargo_mul_model_dict:
        cargo_model_cnt = cargo_mul_model_dict[key]
        cargo_cnt = cargo_cnt + cargo_model_cnt
        
    for key in tot_mul_model_dict:
        try:
            cargo_model_cnt = cargo_mul_model_dict[key]
        except:
            cargo_model_cnt = 0
        
        tot_model_cnt = tot_mul_model_dict[key]
        
        if cargo_model_cnt == tot_model_cnt:
            match_cnt = match_cnt+1
        else: pass
        
        tot_cnt = tot_cnt + tot_model_cnt
        
    common_set_list = list(set(tot_mul_model_list) & set(cargo_mul_model_list))
    if tot_cnt == cargo_cnt:
        if match_cnt == len(common_set_list):
            check_mul_model = True
        else:
            check_mul_model = False
    else :
        check_mul_model = False
    
    return check_mul_model

In [16]:
def _check_mul_year(idx,cargo_mul_car_df,match_df):
    cargo_mul_year_dict = {}
    tot_mul_year = {}
    match_cnt = 0
    tot_cnt = 0
    cargo_cnt = 0
    
    cargo_mul_car_df = cargo_mul_car_df.astype(str)
    match_df = match_df.astype(str)

    cargo_mul_year = cargo_mul_car_df._get_value(idx,'YR')
    tot_mul_year = match_df['YR'].values[0]
    
    cargo_mul_year_list = _json_to_list(cargo_mul_year)
    tot_mul_year_list = _json_to_list(tot_mul_year)

    cargo_mul_year_list = list(map(float,cargo_mul_year_list))
    cargo_mul_year_list = list(map(int,cargo_mul_year_list))
    cargo_mul_year_list = list(map(str,cargo_mul_year_list))
    
    tot_mul_year_list = list(map(float,tot_mul_year_list))
    tot_mul_year_list = list(map(int,tot_mul_year_list))
    tot_mul_year_list = list(map(str,tot_mul_year_list))
    
    cargo_mul_year_dict = _duplicate_cnt_dict(cargo_mul_year_list)
    tot_mul_year_dict = _duplicate_cnt_dict(tot_mul_year_list) 

    for key in cargo_mul_year_dict:
        cargo_year_cnt = cargo_mul_year_dict[key]
        cargo_cnt = cargo_cnt + cargo_year_cnt
        
    for key in tot_mul_year_dict:
        try:
            cargo_year_cnt = cargo_mul_year_dict[key]
        except:
            cargo_year_cnt = 0
        
        tot_year_cnt = tot_mul_year_dict[key]
        
        if cargo_year_cnt == tot_year_cnt:
            match_cnt = match_cnt+1
        else: pass
        
        tot_cnt = tot_cnt + tot_year_cnt
        
    common_set_list = list(set(tot_mul_year_dict) & set(cargo_mul_year_dict))

    if tot_cnt == cargo_cnt:
        if match_cnt == len(common_set_list):
            check_mul_year = True
        else:
            check_mul_year = False
    else :
        check_mul_year = False
    
    return check_mul_year

In [21]:
def _check_mul_acid(idx,cargo_mul_car_df,match_df):
    cargo_mul_car_df = cargo_mul_car_df.astype(str)
    match_df = match_df.astype(str)
    
    cargo_mul_acid = cargo_mul_car_df._get_value(idx,'ACID_NO')
    tot_mul_acid = match_df['ACID NO'].values[0]
    return cargo_mul_acid == tot_mul_acid

In [30]:
def _check_mul_export_id(idx,cargo_mul_car_df,match_df):
    cargo_mul_car_df = cargo_mul_car_df.astype(str)
    match_df = match_df.astype(str)
    
    cargo_mul_export_id = cargo_mul_car_df._get_value(idx,'FREIGHT_FORWARDER_ID')
    tot_mul_export_id = match_df['FOREIGN EXPORTER REGISTRATION NUMBER'].values[0]
    return cargo_mul_export_id == tot_mul_export_id

In [33]:
def _check_mul_import_num(idx,cargo_mul_car_df,match_df):
    cargo_mul_car_df = cargo_mul_car_df.astype(str)
    match_df = match_df.astype(str)
    
    cargo_mul_import_num = cargo_mul_car_df._get_value(idx,'IMPORTER_TAX_NUMBER')
    tot_mul_import_num = match_df['EGYPTIAN IMPORTER VAT NUMBER '].values[0]
    return cargo_mul_import_num == tot_mul_import_num

In [32]:
err_mul_cnt = 0
err_mul_dict = {}
err_mul_chassino_list = []
err_mul_con_list = []
err_mul_model_list = []
err_mul_year_list = []
err_mul_acid_list = []
err_mul_export_id_list = []
err_mul_import_num_list = []

for idx in cargo_mul_car_df.index:
    cargo_mul_chassino = cargo_mul_car_df._get_value(idx,'CHASSINO')
    cargo_mul_acid = cargo_mul_car_df._get_value(idx,'ACID_NO')
    check_chassino,match_df = _check_mul_chassino(cargo_mul_chassino,cargo_mul_acid,tot_mul_car_df)
    
    if check_chassino:
        check_mul_con = _check_mul_con(idx,cargo_mul_car_df,match_df)
        check_mul_model = _check_mul_model(idx,cargo_mul_car_df,match_df)
        check_mul_year = _check_mul_year(idx,cargo_mul_car_df,match_df)
        check_mul_acid = _check_mul_acid(idx,cargo_mul_car_df,match_df)
        check_mul_export_id = _check_mul_export_id(idx,cargo_mul_car_df,match_df)
        check_mul_import_num = _check_mul_import_num(idx,cargo_mul_car_df,match_df)
        
        
    else:
        if len(match_df) > 0:
            print(idx)
        else:
            print('X')
        
    

2


In [200]:
for idx in cargo_one_car_df.index:
    cargo_one_chassino = cargo_one_car_df._get_value(idx,'CHASSINO')
    check_chassino,match_df = _check_chassino(cargo_one_chassino,tot_one_car_df)
    
    if check_chassino:
        check_con = _check_con(idx,cargo_one_car_df,match_df)
        check_model = _check_model(idx,cargo_one_car_df,match_df)
        check_year = _check_year(idx,cargo_one_car_df,match_df)
        check_acid = _check_acid(idx,cargo_one_car_df,match_df)
        check_export_id = _check_export_id(idx,cargo_one_car_df,match_df)
        check_import_num = _check_import_num(idx,cargo_one_car_df,match_df)
        
        any_check = check_con or check_model or check_year or check_acid or check_export_id or check_import_num
        
        print(any_check)
#         print(check_model)
#         print(check_year)
#         print(check_acid)
#         print(check_export_id)
#         print(check_import_num)
        
        
        
            
        
        
        
        
    

True
True


[autoreload of cig_script.compare_and_checker failed: Traceback (most recent call last):
  File "c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "c:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "c:\users\user\appdata\local\programs\python\python38\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\USER\Desktop\CIG\cig_script\compare_and_checker.py", line 3, in <module>
    from config.config_env import Config
ModuleNotFoundError: No module named 'config'
]
